# Simple Procedural Content Generation for Space Engineers

Create functional spaceships in Space Engineers by PCG using the [SE API](https://github.com/iv4xr-project/iv4xr-se-plugin).

Additional information for SE can be found [here](https://github.com/iv4xr-project/iv4xr-se-plugin/tree/main/JvmClient#basic-space-engineers-engine-info).

SE API calls are listed [here](https://github.com/iv4xr-project/iv4xr-se-plugin/blob/main/Source/Ivxr.SePlugin/Communication/AustinJsonRpcSpaceEngineers.cs).

**NOTE**: Have the game *running*; leaving the game simply paused breaks the API (no calls are accepted). *Suggestion*: Use windowed mode, press `Start` after the scenario is loaded and return to this notebook.

Import required packages

In [73]:
import ast
import json
import random
import os
import socket
import time
from typing import List, Optional, Dict, Any, Tuple

## API Interfacing

Variables for the API:

In [22]:
host = 'localhost'
port = 3333

API common structs/classes

In [141]:
def vec2(x: int,
         y: int) -> Dict[str, int]:
    return {
        "X": x,
        "Y": y
    }

def vec3(x: int,
         y: int,
         z: int) -> Dict[str, int]:
    return {
        "X": x,
        "Y": y,
        "Z": z
    }

def vec2f(x: float,
         y: float) -> Dict[str, float]:
    return {
        "X": x,
        "Y": y
    }

def vec3f(x: float,
         y: float,
         z: float) -> Dict[str, float]:
    return {
        "X": x,
        "Y": y,
        "Z": z
    }

def sum_vecs(a: Dict[str, Any],
             b: Dict[str, Any]) -> Dict[str, Any]:
    return {
        "X": a["X"] + b["X"],
        "Y": a["Y"] + b["Y"],
        "Z": a["Z"] + b["Z"],
    }

In [154]:
class Block:
    def __init__(self,
                 block_type: str,
                 position: Dict[str, Any],
                 orientation_forward: Dict[str, Any] = vec3f(0,0,0),
                 orientation_up: Dict[str, Any] = vec3f(0,0,0),
                 relative_position: bool = True):
        self.block_type = block_type
        self.relative_position = relative_position
        self.position = position
        self.orientation_forward = orientation_forward
        self.orientation_up = orientation_up

In [ ]:
character_camera_dist = vec3f((0., 1.6369286, 0.))

API interfacing methods

In [135]:
def generate_json(method: str,
                  params: Optional[List[Any]] = None) -> Dict[str, Any]:
    return {
        "jsonrpc": "2.0",
        "method": method,
        "params": params if params else [],
        "id": random.getrandbits(16)
    }

def compactify_jsons(jsons: List[Dict[str, Any]]) -> List[str]:
    """ JSONs should be single line & compact formatted before calling the API. (@Karel Hovorka)"""
    compacted_jsons = ''
    for j in jsons:
        compacted_jsons += json.dumps(obj=j,
                               separators=(',', ':'),
                               indent=None)
        compacted_jsons += '\r\n'

    return compacted_jsons

# Modified from https://www.binarytides.com/receive-full-data-with-the-recv-socket-function-in-python/
def recv_with_timeout(s: socket.socket,
                      timeout: int = 2):
    #make socket non blocking
    s.setblocking(0)
    #total data partwise in an array
    total_data = [];
    #beginning time
    begin = time.time()
    while True:
        # timeout termination
        if total_data and time.time() - begin > timeout:
            break
        # wait to get data
        elif time.time() - begin > timeout * 2:
            break
        # recv
        try:
            data = s.recv(8192).decode("utf-8")
            if data:
                # early break if socket returns the same data
                if len(total_data) > 1 and data == total_data[-1]:
                    break
                # append new data
                total_data.append(data)
                # change the beginning time for measurement
                begin = time.time()
            else:
                #sleep to indicate a gap
                time.sleep(0.1)
        except:
            # note: Exceptions are thrown due to socket not reading anything before and after data is passed
            pass
    # join all parts to make final string
    return ''.join(total_data)

def call_api(jsons: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    # open socket for communication
    s = socket.socket(family=socket.AF_INET,
                      type=socket.SOCK_STREAM)
    s.connect((host, port))    
    # send methods as compacted JSON bytearray
    s.sendall(compactify_jsons(jsons=jsons).encode('utf-8'))
    # get response
    res = recv_with_timeout(s)
    # close socket
    s.close()
    # due to TCP streming packets, it's possible some JSON-RPC responses are the same;
    # workaround: identify unique JSON-RPC responses by unique id
    return [json.loads(x) for x in list(set(res.strip().split('\r\n')))]

Simple test:

In [136]:
jsons = [
    generate_json(method="Character.TurnOnJetpack"),
    generate_json(method="Character.MoveAndRotate",
                  params={
                    "movement": vec3(10, 0, -1),
                    "rotation3": vec2f(0., 20.),
                    "roll": 0.}
                 )
]

print(call_api(jsons=jsons))

[{'jsonrpc': '2.0', 'result': {'Velocity': {'X': 0.0, 'Y': 0.0, 'Z': 0.0}, 'Extent': {'X': 1.0, 'Y': 1.7999999523162842, 'Z': 1.0}, 'Camera': {'Position': {'X': 25.19757797204165, 'Y': -58.44419341854152, 'Z': -41.43892783281847}, 'OrientationForward': {'X': -0.37182876467704773, 'Y': -0.6148713231086731, 'Z': -0.6954686045646667}, 'OrientationUp': {'X': 0.8788108825683594, 'Y': 0.0081980861723423, 'Z': -0.47709980607032776}}, 'JetpackRunning': True, 'HelmetEnabled': True, 'HealthRatio': 1.0, 'HeadLocalXAngle': 0.0, 'HeadLocalYAngle': 0.0, 'TargetBlock': None, 'TargetUseObject': None, 'Id': 'se0', 'Position': {'X': 23.75538569446207, 'Y': -58.45764706531653, 'Z': -40.65597251262669}, 'OrientationForward': {'X': -0.37182876467704773, 'Y': -0.6148713231086731, 'Z': -0.6954686045646667}, 'OrientationUp': {'X': 0.8788108825683594, 'Y': 0.008198082447052002, 'Z': -0.47709980607032776}}, 'id': 38031}, {'jsonrpc': '2.0', 'result': {'Velocity': {'X': 0.0, 'Y': 0.0, 'Z': 0.0}, 'Extent': {'X': 1

## Block placement

[Blocks DefintionIDs](https://github.com/iv4xr-project/iv4xr-se-plugin/tree/main/JvmClient#definitionid-id) are used to specify which block to place. Here we either ask the API directly to provide us the list of available DefintionIDs or we can load the .JSON which contains them (faster in future runs).

**NOTE**: This cell produces the global variable `block_definitions`, a map between the block type and the API-supported ID to use in the request `params` field.

In [76]:
if not os.path.exists('./block_definitions.json'):
    # poll API for block definition ids
    jsons = [ 
        generate_json(method="Definitions.BlockDefinitions")
        ]
    res = json.loads(call_api(jsons=jsons))
    # transform to map of type:id
    block_definitions = {v['Type']:v for v in [entry['DefinitionId'] for entry in res['result']]}
    with open('./block_definitions.json', 'w') as f:
        json.dump(block_definitions, f)
else:
    with open('./block_definitions.json', 'r') as f:
        block_definitions = json.load(f)

Hardcoded structure (relative positions for each block)

In [176]:
structure = [
    Block('SmallBlockArmorBlock', vec3(0,0,0), vec3(0,0,1), vec3(1,0,0), relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,0,1), vec3(0,0,1), vec3(1,0,0), relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,0,2), vec3(0,0,1), vec3(1,0,0), relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,1,2), vec3(0,0,1), vec3(1,0,0), relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,2,2), vec3(0,0,1), vec3(1,0,0), relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,2,1), vec3(0,0,1), vec3(1,0,0), relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,2,0), vec3(0,0,1), vec3(1,0,0), relative_position=True),
    Block('SmallBlockArmorBlock', vec3(0,1,0), vec3(0,0,1), vec3(1,0,0), relative_position=True),
]

In [178]:
def place_structure(structure):
    # get base position and orientations (player-relative)
    obs = call_api(jsons=[generate_json(method="Observer.Observe")])[0]
    base_position = obs['result']['Position']
#     base_orientation_forward = obs['result']['OrientationForward']    
#     base_orientation_up = obs['result']['OrientationUp']
    # prepare jsons
    jsons = [generate_json(method="Admin.Blocks.PlaceAt",
                           params={
                               "blockDefinitionId": block_definitions[block.block_type],
                               "position": sum_vecs(base_position, block.position) if block.relative_position else block.position,
                               "orientationForward": block.orientation_forward,
                               "orientationUp": block.orientation_up
                           }) for block in structure]
    # produce structure
    call_api(jsons=jsons)

Resulting structure should look like this:

![block_placement_example](./block_placement_example.png)

In [179]:
place_structure(structure)